In [1]:
import os
import tarfile
import requests
import kaggle

OSError: Could not find kaggle.json. Make sure it's located in /home/sam/.kaggle. Or use the environment method.

     |████████████████████████████████| 58 kB 604 kB/s eta 0:00:01
     |████████████████████████████████| 74 kB 428 kB/s eta 0:00:01
     |████████████████████████████████| 78 kB 957 kB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73052 sha256=f8606da8cc1dc807105c9d011799692763c85517e18b284f9a4a0fc2c1e55b8d
  Stored in directory: /home/sam/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6767 sha256=8aebdcdcd462fdb697642e64985f8d3a6f24393939645670376f9a7bf6bf44eb
  Stored in directory: /home/sam/.cache/pip/wheels/91/4d/4f/e740a68c215791688c46c4d6251770a570e8dfea91af1acb5c
Successfully built kaggle python-slugify
Note: you may need to restart the kernel to use updated packages.


In [7]:
DOWNLOAD_ROOT = "https://www.kaggle.com/teajay/global-shark-attacks?select=attacks.csv"
DOWNLOAD_PATH = os.path.join("dataset")
DOWNLOAD_URL = DOWNLOAD_ROOT + "dataset/attacks.zip"

In [19]:
def fetch_data_data(download_url=DOWNLOAD_URL, download_path=DOWNLOAD_PATH):


kaggle.api.authenticate()

kaggle.api.dataset_download_files('The_name_of_the_dataset', path='the_path_you_want_to_download_the_files_to', unzip=True)

In [20]:
fetch_data_data()

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <title>Global Shark Attacks | Kaggle</title>\r\n    <meta charset="utf-8" />\r\n    <meta name="robots" content="index, follow" />\r\n    <meta name="description" content="Data compiled by the global shark attack file" />\r\n    <meta name="turbolinks-cache-control" content="no-cache" />\r\n        <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">\r\n    <meta name="theme-color" content="#008ABC" />\r\n    <script nonce="eAPPxJv398lNhWCK1LnwTQ==" type="text/javascript">\r\n        if (\'serviceWorker\' in navigator) {\r\n            navigator.serviceWorker.getRegistrations()\r\n                .then(function(registrations) {\r\n                    for (let registration of registrations) {\r\n                        registration.unregister();\r\n                    }\r\n                })\r\n                .catch(function(err) {\r\n                    console.error("Service

In [ ]:
def import_load_data(csv_path, encoding="ISO-8859-1"):
    df = pd.read_csv(csv_path, encoding=encoding)
    return df